In [4]:
import sklearn
import pickle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn import svm
from sklearn.linear_model import LinearRegression
from tensorflow.python.keras.utils import losses_utils
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
import math
import os, sys
import requests
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib import style


import tensorflow as tf
from tensorflow.keras import layers
display_settings = {
    'max_columns': 10000,
    'expand_frame_repr': True,  # Развернуть на несколько страниц
    'max_rows': 100,
    'precision': 2,
    'show_dimensions': True
}

for op, value in display_settings.items():
    pd.set_option("display.{}".format(op), value)

# os.chdir(r'C:\Users\nesterenko\OneDrive - API & Interstarch\Desktop\NN\max_1')

# col_name_string = 'Ethanol CBOT, USD/gallon'
# col_name_string = 'Raw Sugar №11'
# col_name_string = 'Wheat MATIF, USD/MT'
# col_name_string = 'Wheat CBOT'
col_name_string = 'Corn CBOT'
# col_name_string = 'US Dollar Index'   #USDBRL
# col_name_string = 'USDBRL'   #USDBRL
# col_name_string = 'Crude Oil WTI'
# col_name_string = 'Soybeans CBOT'
# col_name_string = 'Cotton'
# col_name_string = 'Coffee'
# col_name_string = 'Cococa'
# col_name_string = 'Live Cattle'


my_period = [12]
my_method = [1,2,3,4,5,7]  #1,2,3,4,5,7
my_cutted = [0.1, 0.25, 0.5, 0.75, 0.9]  # 0.1, 0.25, 0.5, 0.75, 0.9
my_pca = [ 0.85, 0.95, 0.97]  # 0.85, 0.95, 0.97

for my_method_i in my_method:
    for my_per_i in my_period:
        for my_cutted_i in my_cutted:
            for my_pca_i in my_pca:
                def conv(df):
                    
                    def add_ratio(i):
                        a2 = i + '_Ratio'    
                        df[a2] = df[col_name_string]/df[i] 
                    def add_change(i):
                        a1 = i + '_ch'
                        df[a1] = df[i].pct_change()*100 
                    
                    def add_change(i):
                        a1 = i + '_ch'
                        df[a1] = df[i].pct_change(periods = 1)*100    
                    def add_change_2(i):
                        a1 = i + '_ch_i'
                        df[a1] = df[i].pct_change(periods = 2)*100 
                    def add_change_3(i):
                        a1 = i + '_ch_i'
                        df[a1] = df[i].pct_change(periods = 2)*100              
                    def add_stoch_20(i):
                        a2 = i + '_stoch'    
                        df[a2] = 2* ((df[i] - df[i].rolling(20).min().shift(1)) / (df[i].rolling(20).max().shift(1) - df[i].rolling(20).min().shift(1)) - 0.5)                                
                    def add_stoch_90(i):
                        a2 = i + '_stoch'    
                        df[a2] = 2* ((df[i] - df[i].rolling(90).min().shift(1)) / (df[i].rolling(90).max().shift(1) - df[i].rolling(90).min().shift(1)) - 0.5)                                
                    def add_stoch_190(i):
                        a2 = i + '_stoch_i'    
                        df[a2] = 2* ((df[i] - df[i].rolling(190).min().shift(1)) / (df[i].rolling(190).max().shift(1) - df[i].rolling(190).min().shift(1)) - 0.5)                                
                    def add_stoch_250(i):
                        a2 = i + '_stoch_ii'    
                        df[a2] = 2* ((df[i] - df[i].rolling(250).min().shift(1)) / (df[i].rolling(250).max().shift(1) - df[i].rolling(250).min().shift(1)) - 0.5)                                
                    def add_mean_two(i):  
                        a2 = i+ "_mean"
                        a3 = i+ "_mean_i"   
                        df[a2] = df[i].rolling(250).mean().shift(1)
                        df[a3] = df[i].rolling(190).mean().shift(1)
                    def add_mean_90(i):    
                        a4 = i+ "_mean_iii"
                        df[a4] = df[i].rolling(90).mean().shift(1)
                    def add_mean_one(i):  
                        a2 = i+ "__mean"
                        a3 = i+ "__mean_i"   
                        a4 = i+ "__mean_ii"
                        df[a2] = df[i].rolling(5).mean().shift(1)
                        df[a3] = df[i].rolling(10).mean().shift(6)
                        df[a4] = df[i].rolling(20).mean().shift(11)   
                    def add_mean_ch_two(i):  
                        a2 = i+ "_mean_ch_"
                        a3 = i+ "_mean_ch__i"   
                        df[a2] = df[i]/df[i].rolling(250).mean().shift(1)
                        df[a3] = df[i]/df[i].rolling(190).mean().shift(1)
                    def add_mean_ch_90(i):    
                        a4 = i+ "_mean_ch__iii"
                        df[a4] = df[i]/df[i].rolling(90).mean().shift(1)                        
                    def add_mean_ch_one(i):  
                        a2 = i+ "__mean_ch_"
                        a3 = i+ "__mean_ch__i"   
                        a4 = i+ "__mean_ch__ii"
                        df[a2] = df[i]/df[i].rolling(5).mean().shift(1)
                        df[a3] = df[i]/df[i].rolling(10).mean().shift(6)
                        df[a4] = df[i]/df[i].rolling(20).mean().shift(11)  

                    def split_by_cat(row):
                        if row > 0:
                            return 1
                        else:
                            return 0
                    df.set_index('Date', inplace = True)

#                     if cut_method_i == 0:
                    c = df.columns
                    for i in c:
                        if i != col_name_string:
                            add_ratio(i)
                        else:
                            add_change(i)
                        
                    if my_method_i == 1:
                        c = df.columns
                        for i in c:  
                            add_stoch_190(i)
                            add_stoch_250(i)
                    if my_method_i == 2:
                        c = df.columns
                        for i in c:  
                            add_mean_ch_two(i)
                    if my_method_i == 3:
                        c = df.columns
                        for i in c:
                            add_stoch_190(i)
                            add_stoch_250(i)
                            add_mean_ch_two(i)
                    if my_method_i == 4: 
                        c = df.columns
                        for i in c:
                            add_stoch_90(i)
                            add_stoch_190(i)
                            add_stoch_250(i)
                            add_mean_ch_90(i)
                            add_mean_ch_two(i)  
                    if my_method_i == 5:
                        c = df.columns
                        for i in c:  
                            add_stoch_20(i)
                            add_stoch_90(i)
                            add_stoch_190(i)
                            add_stoch_250(i)
                            add_mean_ch_one(i)
                            add_mean_ch_two(i) 
                            add_mean_ch_90(i) 
                    if my_method_i == 6:
                        c = df.columns
                        for i in c:  
                            add_mean_one(i)
                            add_mean_two(i)
                            
                    if my_method_i == 7:
                        c = df.columns
                        for i in c:  
                            add_change(i)
                            add_change_2(i)
                            add_change_3(i)   
                        df = df.replace([np.inf, -np.inf], np.nan)
                        df.fillna( method ='ffill', inplace = True)
                        c = df.columns
                        for i in c: 
                            add_stoch_190(i)
                            add_stoch_250(i)
                            add_mean_ch_two(i)


                    df['prelabel'] = df[col_name_string].rolling(my_per_i).mean().shift(-my_per_i)
                    df['label'] = df['prelabel'] / df[col_name_string] * 100.0 - 100.0
                    df.drop(columns =['prelabel'], inplace = True) 

                    df['cat'] = df['label'].apply(lambda row: split_by_cat(row))
                    df['label'] = df['cat'] 
                    df.drop(columns =['cat'], inplace = True) 
                    df = df.drop(list(df)[0:cn], axis=1)  
                    return df
                df = pd.read_excel(open('weekly_av_prices.xlsx', 'rb'), skiprows= 0,  header = [0], parse_dates = True )
                cvc = df.columns[1:]
                cn = cvc.shape[0]
                df = conv(df)
                df_fin = df.copy()
                
                df_fin = df_fin.replace([np.inf, -np.inf], np.nan)
                df_fin.fillna( method ='ffill', inplace = True) 



                my_sh = df_fin.shape[1]
                my_cu = int(round(my_sh*my_cutted_i,0))

                print(my_cu)


                from sklearn.feature_selection import SelectKBest
                from sklearn.feature_selection import f_classif

                df = df_fin.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                X = scale_X.transform(X)
                y = np.array(df['label'])

                selector = SelectKBest(f_classif, k= my_cu)
                selector.fit(X, y)
                cols = selector.get_support(indices=True)
                print(cols)
                df_cutted = df_fin.iloc[:, cols]
                print(df_cutted.columns)
                df_cutted["label"] = df_fin["label"].copy()


                df = df_cutted.loc['2014':'2018'].copy()        
                X = np.array(df.drop(['label'], 1))        
                scale_X = StandardScaler()
                scale_X.fit(X)
                X = scale_X.transform(X)        


                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_pca_i)
                pca.fit(X)
                my_dim  =  pca.n_components_

                print("my_cu",my_cu)
                print("my_pca_i", my_pca_i)
                print("X,shape", X.shape)       
                print("Just dim", my_dim)


                ## GPC BOOST

                from sklearn.gaussian_process import GaussianProcessClassifier
                from sklearn.gaussian_process.kernels import RBF
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                from sklearn.model_selection import StratifiedShuffleSplit 
                cv = StratifiedShuffleSplit(n_splits=3,  random_state=42)


                kernel = 1.0 * RBF(1.0)
                clf_GPC = GaussianProcessClassifier(kernel=kernel, random_state=0)


                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                print(my_dim)
                print(X_train.shape)
                #1
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)
                y_train_bin = y_train
                y_test_bin = y_test   
#                 y_train_bin = tf.keras.utils.to_categorical(y_train)
#                 y_test_bin = tf.keras.utils.to_categorical(y_test)


                clf_GPC.fit(X_train, y_train_bin)
                p = clf_GPC.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)


                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])
                y_bin = y
                X = pca.transform(X)
                # clf_RF.fit(X, y_bin)
                p = clf_GPC.predict(X)
                gpc_acc = accuracy_score(y_bin, p)
                my_precision = precision_score(y_bin, p, average='micro')
                my_recall = recall_score(y_bin, p, average='micro')
                my_f1 = f1_score(y_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("my_accuracy:", round(gpc_acc, 2),"my_precision:", round(my_precision,2), "my_recall:", round(my_recall,2) , "my_f1:", round(my_f1,2))
                print(my_cm)

                
                ## GB

                from sklearn.ensemble import GradientBoostingClassifier
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                from sklearn.model_selection import StratifiedShuffleSplit 
                cv = StratifiedShuffleSplit(n_splits=3,  random_state=42)
          
                clf_GB = GradientBoostingClassifier(random_state=0)

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                print(my_dim)
                print(X_train.shape)
                #1
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)
                y_train_bin = y_train
                y_test_bin = y_test   
#                 y_train_bin = tf.keras.utils.to_categorical(y_train)
#                 y_test_bin = tf.keras.utils.to_categorical(y_test)


                clf_GB.fit(X_train, y_train_bin)
                p = clf_GB.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)


                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])
                y_bin = y
                X = pca.transform(X)
                # clf_RF.fit(X, y_bin)
                p = clf_GB.predict(X)
                GB_acc = accuracy_score(y_bin, p)
                my_precision = precision_score(y_bin, p, average='micro')
                my_recall = recall_score(y_bin, p, average='micro')
                my_f1 = f1_score(y_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("my_accuracy:", round(GB_acc, 2),"my_precision:", round(my_precision,2), "my_recall:", round(my_recall,2) , "my_f1:", round(my_f1,2))
                print(my_cm)                
                
                ## NS

                from sklearn.svm import NuSVC
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                from sklearn.model_selection import StratifiedShuffleSplit 
                cv = StratifiedShuffleSplit(n_splits=3,  random_state=42)
          
                clf_NS = NuSVC(probability=True)

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                print(my_dim)
                print(X_train.shape)
                #1
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)
                y_train_bin = y_train
                y_test_bin = y_test   
#                 y_train_bin = tf.keras.utils.to_categorical(y_train)
#                 y_test_bin = tf.keras.utils.to_categorical(y_test)


                clf_NS.fit(X_train, y_train_bin)
                p = clf_NS.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)


                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])
                y_bin = y
                X = pca.transform(X)
                # clf_RF.fit(X, y_bin)
                p = clf_NS.predict(X)
                NS_acc = accuracy_score(y_bin, p)
                my_precision = precision_score(y_bin, p, average='micro')
                my_recall = recall_score(y_bin, p, average='micro')
                my_f1 = f1_score(y_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("my_accuracy:", round(NS_acc, 2),"my_precision:", round(my_precision,2), "my_recall:", round(my_recall,2) , "my_f1:", round(my_f1,2))
                print(my_cm)                                
                
                ## LDA

                from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                from sklearn.model_selection import StratifiedShuffleSplit 
                cv = StratifiedShuffleSplit(n_splits=3,  random_state=42)
          
                clf_LDA = LDA()

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                print(my_dim)
                print(X_train.shape)
                #1
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)
                y_train_bin = y_train
                y_test_bin = y_test   
#                 y_train_bin = tf.keras.utils.to_categorical(y_train)
#                 y_test_bin = tf.keras.utils.to_categorical(y_test)


                clf_LDA.fit(X_train, y_train_bin)
                p = clf_LDA.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)


                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])
                y_bin = y
                X = pca.transform(X)
                # clf_RF.fit(X, y_bin)
                p = clf_LDA.predict(X)
                LDA_acc = accuracy_score(y_bin, p)
                my_precision = precision_score(y_bin, p, average='micro')
                my_recall = recall_score(y_bin, p, average='micro')
                my_f1 = f1_score(y_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("my_accuracy:", round(LDA_acc, 2),"my_precision:", round(my_precision,2), "my_recall:", round(my_recall,2) , "my_f1:", round(my_f1,2))
                print(my_cm)                
                
                
                ## QDA

                from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                from sklearn.model_selection import StratifiedShuffleSplit 
                cv = StratifiedShuffleSplit(n_splits=3,  random_state=42)
          
                clf_QD = QuadraticDiscriminantAnalysis()

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                print(my_dim)
                print(X_train.shape)
                #1
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)
                y_train_bin = y_train
                y_test_bin = y_test   
#                 y_train_bin = tf.keras.utils.to_categorical(y_train)
#                 y_test_bin = tf.keras.utils.to_categorical(y_test)


                clf_QD.fit(X_train, y_train_bin)
                p = clf_QD.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)


                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])
                y_bin = y
                X = pca.transform(X)
                # clf_RF.fit(X, y_bin)
                p = clf_QD.predict(X)
                QD_acc = accuracy_score(y_bin, p)
                my_precision = precision_score(y_bin, p, average='micro')
                my_recall = recall_score(y_bin, p, average='micro')
                my_f1 = f1_score(y_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_bin, p)  #.argmax(axis=1) .argmax(axis=1)
                print("my_accuracy:", round(QD_acc, 2),"my_precision:", round(my_precision,2), "my_recall:", round(my_recall,2) , "my_f1:", round(my_f1,2))
                print(my_cm)


                ## Stochastic Gradient Desscent

                from sklearn.linear_model import SGDClassifier
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                from sklearn.model_selection import StratifiedShuffleSplit 
                from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

                # df_cutted = df_cuttedby_rf.copy()
                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                X = scale_X.fit_transform(X)
                y = np.array(df['label'])


                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                2#
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)



                params = {
                    "max_iter" : [100, 500, 1000, 2000],
                    "loss" : ["log",  "modified_huber"],   #"hinge", "log", "squared_hinge", "modified_huber"
                    "alpha" : [0.0001, 0.001, 0.01, 0.1, 0.15],
                    "penalty" : ["l2", "l1", "none"],
                }

                model = SGDClassifier() #max_iter=1000
                ovo_clf = GridSearchCV(model, param_grid=params)


                y_train_bin = y_train
                y_test_bin = y_test

                ovo_clf.fit(X_train, y_train_bin)
                predictions = ovo_clf.predict(X_test)

                my_accuracy = accuracy_score(y_test_bin, predictions)
                my_precision = precision_score(y_test_bin, predictions, average='micro')
                my_recall = recall_score(y_test_bin, predictions, average='micro')
                my_f1 = f1_score(y_test_bin, predictions, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, predictions)   #.argmax(axis=1)
                print("my_accuracy:", round(my_accuracy, 2),"my_precision:", round(my_precision,2), "my_recall:", round(my_recall,2) , "my_f1:", round(my_f1,2))
                print(my_cm)

                # df = df_fin.loc['2019'].copy()
                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])
                y_bin = y
                X = pca.transform(X)
                p = ovo_clf.predict(X)

                sgd_acc = accuracy_score(y_bin, p)
                my_precision = precision_score(y_bin, p, average='micro')
                my_recall = recall_score(y_bin, p, average='micro')
                my_f1 = f1_score(y_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_bin, p)   #.argmax(axis=1)
                print("my_accuracy:", round(sgd_acc, 2),"my_precision:", round(my_precision,2), "my_recall:", round(my_recall,2) , "my_f1:", round(my_f1,2))
                print(my_cm)
                print(ovo_clf.best_params_)
                # pr = ovo_clf.predict_proba(X)
                # pr

                ## KNeighbors

                from sklearn.neighbors import KNeighborsClassifier
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                from sklearn.model_selection import StratifiedShuffleSplit 
                from sklearn.inspection import permutation_importance
                # cv = StratifiedShuffleSplit(n_splits=3,  random_state=42)

                #     param_grid = [{ 'max_depth' : [ 3, 6]}]
                #     forest_reg = DecisionTreeClassifier(random_state=42)
                #     model = GridSearchCV(forest_reg, param_grid, cv=3 , scoring= 'roc_auc' )

                #     param_grid = [{ 'n_estimators' : [ 3 , 10, 200 ], 'max_features': [ 2 , 4 , 6 , 8 ]}, {'bootstrap':[False], 'n_estimators' : [ 3 , 10 ], 'max_features' : [ 2 , 3 , 4 ]}]


                k_range = list(range(1, 35))
                param_grid = dict(n_neighbors=k_range)

                # param_grid = { 'n_estimators': [500], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth' : [7], 'criterion' :['gini', 'entropy']}
                forest_reg = KNeighborsClassifier()
                kn_RF = GridSearchCV(forest_reg, param_grid, cv=3 , scoring= 'accuracy' ) 

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                X = scale_X.fit_transform(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                3#
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)
                # y_train_bin = y_train
                # y_test_bin = y_test   
                y_train_bin = tf.keras.utils.to_categorical(y_train)
                y_test_bin = tf.keras.utils.to_categorical(y_test)


                kn_RF.fit(X_train, y_train_bin)
                p = kn_RF.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin.argmax(axis=1), p.argmax(axis=1))  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)

                print(kn_RF.best_params_)


                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])
                y =  tf.keras.utils.to_categorical(y)
                X = pca.transform(X)
                p = kn_RF.predict(X)
                kn_acc = accuracy_score(y, p)
                my_precision = precision_score(y, p, average='micro')
                my_recall = recall_score(y, p, average='micro')
                my_f1 = f1_score(y, p, average='micro') # , average='micro'
                my_cm = confusion_matrix(y.argmax(axis=1), p.argmax(axis=1))  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)



                ## Logistics Regression

                from sklearn.linear_model import LogisticRegression
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                grid={"C":np.logspace(-4,-2,25), "penalty":["l1","l2"]}# l1 lasso l2 ridge
                logreg=LogisticRegression()
                logreg_cv=GridSearchCV(logreg,grid, cv=3)

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                X = scale_X.fit_transform(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                4#
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)

                y_train_bin = y_train
                y_test_bin = y_test  

                logreg_cv.fit(X_train, y_train)
                p = logreg_cv.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p , average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, p )  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)

                print(logreg_cv.best_params_)

                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])

                X = pca.transform(X) 
                p = logreg_cv.predict(X)
                lg_acc = accuracy_score(y, p)
                my_precision = precision_score(y, p, average='micro')
                my_recall = recall_score(y, p, average='micro')
                my_f1 = f1_score(y, p , average='micro') # , average='micro'
                my_cm = confusion_matrix(y, p )  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(lg_acc, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)
                print(logreg_cv.best_params_)

                ## SVC

                from sklearn.svm import SVC
                from sklearn.model_selection import GridSearchCV  
                from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve
                # param_grid = {'C': [0.1, 1, 10, 100, 1000],  
                #               'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
                #               'kernel': ['rbf' ,"poly"]} 

                param_grid = { 'kernel': ['rbf' ,"poly"]} 
                svc_clf=SVC(probability=True)
                svc_cv=GridSearchCV(svc_clf, param_grid, cv=3)

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                X = scale_X.fit_transform(X)
                y = np.array(df['label'])

                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                5#
                pca.fit(X_train)
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 
                print(pca.n_components_)

                y_train_bin = y_train
                y_test_bin = y_test  

                svc_cv.fit(X_train, y_train)
                p = svc_cv.predict(X_test)
                my_accuracy = accuracy_score(y_test_bin, p)
                my_precision = precision_score(y_test_bin, p, average='micro')
                my_recall = recall_score(y_test_bin, p, average='micro')
                my_f1 = f1_score(y_test_bin, p , average='micro') # , average='micro'
                my_cm = confusion_matrix(y_test_bin, p )  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(my_accuracy, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)

                print(svc_cv.best_params_)

                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                y = np.array(df['label'])

                X = pca.transform(X) 
                p = svc_cv.predict(X)
                svc_acc = accuracy_score(y, p)
                my_precision = precision_score(y, p, average='micro')
                my_recall = recall_score(y, p, average='micro')
                my_f1 = f1_score(y, p , average='micro') # , average='micro'
                my_cm = confusion_matrix(y, p )  #.argmax(axis=1) .argmax(axis=1)
                print("Accuracy:", round(svc_acc, 2),"Precision:", round(my_precision,2), "Recall:", round(my_recall,2) , "f1:", round(my_f1,2))
                print(my_cm)


                # Neural Networks 2014-2018

                from tensorflow import keras
                from keras.wrappers.scikit_learn import KerasClassifier

                df = df_cutted.loc['2014':'2018'].copy()
                # df = df_cutted.loc['2014':'2018']
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                X = scale_X.transform(X)
                y = np.array(df['label'])
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)
                pca = PCA(n_components = my_dim)
                6#
                pca.fit(X_train) 
                X_train = pca.transform(X_train) 
                X_test = pca.transform(X_test) 

                y_train_bin = y_train
                y_test_bin = y_test

        #         y_train_bin = tf.keras.utils.to_categorical(y_train)
        #         y_test_bin = tf.keras.utils.to_categorical(y_test)
                a = X_train.shape[1]

                def create_model():
                    model = tf.keras.Sequential()
                    model._estimator_type = "classifier"
                    model.add(layers.Dense(a, input_dim=a, activation='relu'))
                #     model.add(layers.Dense(a/2, input_dim=a, activation='relu'))
                    model.add(layers.Dense(2, activation='softmax'))
                    model.compile(loss='sparse_categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])
                    return model

                Kmodel = KerasClassifier(build_fn=create_model, verbose=0)
                Kmodel._estimator_type = "classifier"

                batch_size = [50, 300, 500]
                epochs = [50, 100, 300]
                param_grid = dict(batch_size=batch_size, epochs=epochs)
                grid = GridSearchCV(estimator=Kmodel, param_grid=param_grid, cv=3)
                nn = grid.fit(X_train, y_train_bin)

                print("Best: %f using %s" % (nn.best_score_, nn.best_params_))
                means = nn.cv_results_['mean_test_score']
                stds = nn.cv_results_['std_test_score']
                params = nn.cv_results_['params']
                for mean, stdev, param in zip(means, stds, params):
                    print("%f (%f) with: %r" % (mean, stdev, param))

                ## NN 2019

                batch_size_i = nn.best_params_["batch_size"]
                epochs_i = nn.best_params_["epochs"]
                model = create_model()
                model.fit(X_train, y_train_bin, epochs=epochs_i, batch_size=batch_size_i)
        #         history = model.fit(X_train, y_train_bin, epochs=epochs_i, batch_size=batch_size_i)
        #         plt.plot(history.history['accuracy'])
        #         plt.ylabel('accuracy')
        #         plt.xlabel('epoch')
        #         plt.show()

                scores = model.evaluate(X_test, y_test_bin)
                first_score = round(scores[1]*100 , 2)
                print(first_score)

                df = df_cutted.loc['2019'].copy()
                # df = df_cutted.loc['2019']
                X = np.array(df.drop(['label'], 1))
                y = np.array(df['label'])
                y_bin = tf.keras.utils.to_categorical(y)

                # s=[]
                # n = len(y_bin)
                # for i in range(n):
                #     s.append(y_test_bin[i][1])
                # y_bin = s 


                # y =  tf.keras.utils.to_categorical(y)
                X = scale_X.transform(X)
                X = pca.transform(X)
                print(X.shape)
                p = model.predict_classes(X)

                my_cm = confusion_matrix(y, p) 
                print(my_cm)
                # scores = model.evaluate(p, y_bin)
                # first_score = round(scores[1]*100 , 2)
                # print(first_score)

                # s = []
                # l = len(y)
                # for i in range(0, l):
                #     s.append(i)
                # print(s)
                # print(len(s))
                # print(len(y))
                # plt.plot( p , "g-", linestyle = 'dotted', linewidth = 6, color = 'darkblue' ) 
                # plt.scatter( s, y , color='black') 
                # plt.scatter( s, p, color='red') 

                nn_acc = accuracy_score(y, p)
                print(nn_acc)
                # print(y)
                # print(p)
                # print(my_cm[0,1])

                ## ensemble         
                
                from sklearn.ensemble import VotingClassifier 
                voting_clf = VotingClassifier(
                    estimators= [ ("SGD_ovo", ovo_clf), ("KN", kn_RF), ( 'lr', logreg_cv), ('svc', svc_cv), ("nn", nn), ( 'GPC', clf_GPC), ( 'QD', clf_QD),  ( 'GB', clf_GB), ( 'LDA', clf_LDA),  ( 'NS', clf_NS)], 
                    voting= 'hard' )

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                X = scale_X.transform(X)
                print(X.shape)
                y = np.array(df['label'])

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                pca.fit(X)
                X = pca.transform(X) 
                print(X.shape)

                voting_clf.fit(X, y) 

                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                X = pca.transform(X) 
                y = np.array(df['label'])

                p = voting_clf.predict(X)
                
                
                my_cm = confusion_matrix(y, p) 
                print(my_cm)
                hard_acc = accuracy_score(y, p)
                print(hard_acc)


                from sklearn.ensemble import VotingClassifier 
                voting_clf = VotingClassifier(
                    estimators= [ ("SGD_ovo", ovo_clf), ("KN", kn_RF), ( 'lr', logreg_cv), ('svc', svc_cv), ("nn", nn), ( 'GPC', clf_GPC), ( 'QD', clf_QD),  ( 'GB', clf_GB), ( 'LDA', clf_LDA),  ( 'NS', clf_NS)], 
                    voting= 'soft' )

                df = df_cutted.loc['2014':'2018'].copy()
                X = np.array(df.drop(['label'], 1))
                scale_X = StandardScaler()
                scale_X.fit(X)
                X = scale_X.transform(X)
                print(X.shape)
                y = np.array(df['label'])

                from sklearn.decomposition import PCA
                pca = PCA(n_components = my_dim)
                pca.fit(X)
                X = pca.transform(X) 
                print(X.shape)

                voting_clf.fit(X, y) 

                df = df_cutted.loc['2019'].copy()
                X = np.array(df.drop(['label'], 1))
                X = scale_X.transform(X)
                X = pca.transform(X) 
                y = np.array(df['label'])


#                 print(X)
                p = voting_clf.predict(X)
                my_cm = confusion_matrix(y, p) 
                print(my_cm)
                soft_acc = accuracy_score(y, p)
                print(soft_acc)                
                pr = voting_clf.predict_proba(X)
                
                my_threshold = 0.65
                my_hthreshold = 0.75
                pp=[]
                ppp = []
                yy = []
                yyy = []
                s = []
                ss = []
                n = len(y)
                for i in range(n):
                    a = pr[i][0]
                    b = pr[i][1]
                    z = max(a,b)
                    s.append(z) 
                    if z > my_threshold:
                            pp.append(p[i])
                            yy.append(y[i])
                    if z > my_hthreshold:
                            ppp.append(p[i])
                            yyy.append(y[i])                            
                soft_acc_th = accuracy_score(yy, pp)
                print("Soft_acc_th", soft_acc_th)
                soft_acc_hth = accuracy_score(yyy, ppp)
                print("Soft_acc_hth", soft_acc_hth)
                l = len(pp)
                ll=len(ppp)
                f = open('ensemble_register.txt', 'a')
                f.write("MARKET" + "^" + str(col_name_string) + "^" +
                        "my_per_i" + "^" + str(my_per_i)  + "^" +
                        "my_method_i" + "^" + str(my_method_i)  + "^" +
                        "my_cutted_i" + "^" + str(my_cutted_i)  + "^" +
                        "my_pca_i"  + "^" + str(my_pca_i) + "^" + 
                        "gpc_acc"  + "^" + str(gpc_acc) + "^" +
                        "QD_acc"  + "^" + str(QD_acc) + "^" +
                        "GB_acc"  + "^" + str(GB_acc) + "^" +
                        "LDA_acc"  + "^" + str(LDA_acc) + "^" +
                        "NS_acc"  + "^" + str(NS_acc) + "^" +
                        "sgd_acc"  + "^" + str(sgd_acc) + "^" +
                        "kn_acc"  + "^" + str(kn_acc) + "^" +
                        "lg_acc"  + "^" + str(lg_acc) + "^" +
                        "svc_acc"  + "^" + str(svc_acc) + "^" +
                        "nn_acc"  + "^" + str(nn_acc) + "^" +        
                        "hard_acc" + "^" + str(hard_acc) + "^" +
                        "soft_acc" + "^" + str(soft_acc)  + "^" +
                        "soft_acc_th" + "^" + str(soft_acc_th) + "^" +
                        "soft_acc_hth" + "^" + str(soft_acc_hth) + "^" +
                        "l" + "^" + str(l)  + "^" +
                        "ll" + "^" + str(ll)  + "^" +  "\n")
                f.close                

11
[ 3  4  6  9 13 18 42 72 80 90 99]
Index(['US Dollar Index_Ratio', 'Eurodollar_Ratio', 'AUDUSD_Ratio',
       'Gold_Ratio', 'Soybean Oil CBOT_Ratio', 'Cotton_Ratio',
       'Corn CBOT_stoch_ii', 'Eurodollar_Ratio_stoch_ii',
       'SP500_Ratio_stoch_ii', 'Soybean Oil CBOT_Ratio_stoch_ii',
       'Cotton_Ratio_stoch_i'],
      dtype='object')
my_cu 11
my_pca_i 0.85
X,shape (260, 11)
Just dim 4
4
(247, 11)
4
Accuracy:

C:\Users\nesterenko\anaconda3\lib\site-packages\ipykernel_launcher.py:234: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


 0.62 Precision: 0.62 Recall: 0.62 f1: 0.62
[[3 4]
 [1 5]]
my_accuracy: 0.49 my_precision: 0.49 my_recall: 0.49 my_f1: 0.49
[[ 0 27]
 [ 0 26]]
4
(247, 11)
4
Accuracy: 0.77 Precision: 0.77 Recall: 0.77 f1: 0.77
[[4 3]
 [0 6]]
my_accuracy: 0.62 my_precision: 0.62 my_recall: 0.62 my_f1: 0.62
[[ 7 20]
 [ 0 26]]
4
(247, 11)
4
Accuracy: 0.62 Precision: 0.62 Recall: 0.62 f1: 0.62
[[4 3]
 [2 4]]
my_accuracy: 0.49 my_precision: 0.49 my_recall: 0.49 my_f1: 0.49
[[ 0 27]
 [ 0 26]]
4
(247, 11)
4
Accuracy: 0.69 Precision: 0.69 Recall: 0.69 f1: 0.69
[[4 3]
 [1 5]]
my_accuracy: 0.49 my_precision: 0.49 my_recall: 0.49 my_f1: 0.49
[[ 0 27]
 [ 0 26]]
4
(247, 11)
4
Accuracy: 0.62 Precision: 0.62 Recall: 0.62 f1: 0.62
[[3 4]
 [1 5]]
my_accuracy: 0.51 my_precision: 0.51 my_recall: 0.51 my_f1: 0.51
[[27  0]
 [26  0]]
4
my_accuracy: 0.62 my_precision: 0.62 my_recall: 0.62 my_f1: 0.62
[[3 4]
 [1 5]]
my_accuracy: 0.53 my_precision: 0.53 my_recall: 0.53 my_f1: 0.53
[[27  0]
 [25  1]]
{'alpha': 0.1, 'loss': 'log

C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit faile

C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1304, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 443, in _check_solver
    "got %s penalty." % (solver, penalty))
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
C:\Users\nesterenko\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit faile

Accuracy: 0.62 Precision: 0.62 Recall: 0.62 f1: 0.62
[[3 4]
 [1 5]]
{'C': 0.0031622776601683794, 'penalty': 'l2'}
Accuracy: 0.55 Precision: 0.55 Recall: 0.55 f1: 0.55
[[27  0]
 [24  2]]
{'C': 0.0031622776601683794, 'penalty': 'l2'}
4
Accuracy: 0.69 Precision: 0.69 Recall: 0.69 f1: 0.69
[[3 4]
 [0 6]]
{'kernel': 'rbf'}
Accuracy: 0.51 Precision: 0.51 Recall: 0.51 f1: 0.51
[[ 2 25]
 [ 1 25]]


KeyboardInterrupt: 

In [ ]:
                print("Average proba", np.sum(s)/len(s))